In [1]:
# -*- coding: utf-8 -*-
import os
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
from bs4 import element
from itertools import product
import numpy as np
import pandas as pd
from abbreviations import schwartz_hearst
import subprocess
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/rds/general/user/yh4218/home/.conda/envs/NLP/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
base_dir = '/rdsgpfs/general/user/yh4218/home/data/NLP'
file_list = []
def get_files(base_dir,file_list):
    files = os.listdir(base_dir)
    for i in files:
        abs_path = os.path.join(base_dir,i)
        if re.match(r'(.*)PMC(.*).html',abs_path):
            file_list.append(abs_path)
        elif os.path.isdir(abs_path)&('ipynb_checkpoints' not in abs_path):
            get_files(abs_path,file_list)
    return file_list

file_list = get_files(base_dir,file_list)

In [3]:
len(file_list)

4366

# All unique H2 heading

In [4]:
def extract_all_unique_h2_headings(heading_text_list,soup_og,file_name):
    headings = soup_og.find_all('h2')
    for number, heading in enumerate(headings):
        heading_text =  heading.get_text()
        if heading_text not in (heading_texts.split('\t\t')[0] for heading_texts in heading_text_list):
            heading_text_list.append(heading_text+'\t\t'+file_name.split('/')[-1])
    return

def extract_all_h2_headings(heading_text_list,heading_text_list_with_filename,soup_og,file_name):
    headings = soup_og.find_all('h2')
    for number, heading in enumerate(headings):
        heading_text =  heading.get_text()
        heading_text_list.append(''.join([i.lower() for i in heading_text if not i.isdigit()]).strip('.').strip(' ').strip(':').strip(' ').replace('\n',' '))
        heading_text_list_with_filename.append(''.join([i.lower() for i in heading_text if not i.isdigit()]).strip('.').strip(' ').strip(':').strip(' ').replace('\n',' ')+'thereisaspace'+file_name.split('/')[-1])
    return

def extract_all_h2_headings(heading_text_list,heading_text_list_with_filename,soup_og,file_name):
    headings = soup_og.find_all('h2')
    for number, heading in enumerate(headings):
        heading_text =  heading.get_text()
        heading_text_list.append(''.join([i.lower() for i in heading_text if not i.isdigit()]).strip('.').strip(' ').strip(':').strip(' ').replace('\n',' '))
        heading_text_list_with_filename.append(''.join([i.lower() for i in heading_text if not i.isdigit()]).strip('.').strip(' ').strip(':').strip(' ').replace('\n',' ')+'thereisaspace'+file_name.split('/')[-1])
    return

# all

In [5]:
heading_text_list=[]
heading_text_list_with_filename=[]
for file in file_list:
    #output_file.write(file+'\n')
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        extract_all_h2_headings(heading_text_list,heading_text_list_with_filename,soup,file)

In [6]:
heading_text_list_unique=[]
heading_text_list_unique_for_count=[]
for number,heading in enumerate(heading_text_list):
    number_of_heading = heading_text_list.count(heading)
    if number_of_heading >1 and heading_text_list_unique_for_count.count(heading)<1: 
        heading_text_list_unique.append(heading_text_list_with_filename[number]+'\t'+str(number_of_heading))
        heading_text_list_unique_for_count.append(heading)

In [7]:
heading_text_list_unique_for_count

['associated data',
 'abstract',
 'introduction',
 'results',
 'discussion',
 'materials and methods',
 'conclusions',
 'supplementary materials',
 'author contributions',
 'funding',
 'conflicts of interest',
 'references',
 'formats',
 'share',
 'acknowledgments',
 'methods',
 'patents',
 'appendix a',
 'results and discussion',
 'experimental section',
 'conflict of interest',
 'lc-ms measurements',
 'overview',
 'the application in early diagnosis of pancreatic cancer',
 'the application in early diagnosis of kidney cancer',
 'acknowledgements',
 'material and methods',
 'abbreviations',
 'method',
 'authors contributions',
 'supplementary material',
 'coenzymes analysis in tissue',
 'coenzymes in human whole blood',
 'subcellular coenzymes',
 'supplementary files',
 'acknowledgments and disclaimer',
 'appendix a. uncertainty analysis',
 'author copyright statement',
 'experimental',
 'references and notes',
 'subjects and methods',
 'experimental methods',
 'data availability',
 '

In [111]:
with open('/rdsgpfs/general/user/yh4218/home/data/all_unique_h2_headings_with_counts.txt','w') as f:
    for heading in heading_text_list_unique:
        f.write(str(heading)+'\n')

# unique

In [32]:
heading_text_list=[]
for file in file_list:
    #output_file.write(file+'\n')
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        extract_all_unique_h2_headings(heading_text_list,soup,file)

In [33]:
with open('/rdsgpfs/general/user/yh4218/home/data/all_unique_h2_headings.txt','w') as f:
    for heading in heading_text_list:
        f.write(str(heading)+'\n')

# Find not classified heading

In [19]:
discard_list=['funding',
              'Conflicts of Interest',
              #'Additional Information',
              'Acknowledgment',
              'Acknowledgement',
              'financial',
              'contribution',
              'conflict',
              'footnotes',
              'interest',
              'Reference',
              'Formats',
              'Share',
              'Patent',
              'disclosure',
              'Availability',
              'additional']
abstract_list=['abstract']
introduction_list=['introduction']
background_list=['background']
overview_list=['overview']
supplementary_material_list=['supplementary','Associated Data','Supplemental']
abbreviation_list=['glossary','abbreviation']
method_material_list=['method','material','Experimental'] # sometimes titled Experimental section & procedure 
result_discussion_list=['Results','discussion']
conclusion_list=['conclusion']  # sometimes with future direction
all_list=discard_list+abstract_list+introduction_list+background_list+overview_list+supplementary_material_list+abbreviation_list+method_material_list+result_discussion_list+conclusion_list
newlist=[]
match=[]
for heading in heading_text_list:
    #newlist.append(heading.split('\t')[0])
    newlist.append(heading)
with open('/rdsgpfs/general/user/yh4218/home/outputs/all_unique_h2_headings.txt','w') as output_file:
    for heading in newlist:
        if not any(word.lower() in heading.lower() for word in all_list):
            match.append(heading)
for m in match:print (m)

Appendix A	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC6724101.html
4. LC-MS Measurements	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC5618317.html
Appendix A. Uncertainty Analysis	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC6918257.html
Author Copyright Statement	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC6918257.html
2. The Application in Early Diagnosis of Pancreatic Cancer	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC3901290.html
3. The Application in Early Diagnosis of Kidney Cancer	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC3901290.html
2. Coenzymes Analysis in Tissue	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC6027050.html
3. Coenzymes in Human Whole Blood	/rdsgpfs/general/user/yh4218/home/data/NLP-MWAS/Metabolites (Journal)/PMC6027050.html
4. Subcellular Coenzymes	/rdsgpfs/gener

In [8]:
fuzz.ratio("Acknowledgement","Acknowledgment")
fuzz.partial_ratio("Acknowledgement","Acknowledgment")

93